In [327]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import math
import random
import pandas as pd


In [447]:
def load_dataset(path, dataset, method):  #method 2 for matrix loading
    with open(path, 'r', newline='\n') as csvfile:
        reader = csv.reader(csvfile,  quoting=csv.QUOTE_NONNUMERIC)
        if method ==2:
            for line in reader:
                dataset.append(line[:len(line)-1])
        elif method ==1 :
            for line in reader:
                dataset.append(line[:len(line)-1])
pos = []
neg =[]
neg_mean= []
pos_mean=[]
cov_mat= []
path = './hwk2_datasets/DS1_m_0.txt'
load_dataset(path, neg_mean,1)
neg_mean = neg_mean[0]
path = './hwk2_datasets/DS1_m_1.txt'
load_dataset(path, pos_mean,1)
pos_mean= pos_mean[0]
path = './hwk2_datasets/DS1_Cov.txt'
load_dataset(path, cov_mat, 2)

print(len(pos_mean))
#print(cov_mat)


20


In [448]:
# Generation of multivariate Gaussian datapoints
np.random.seed(2)
neg = np.random.multivariate_normal(neg_mean, cov_mat, 2000).T
pos = np.random.multivariate_normal(pos_mean, cov_mat, 2000).T
pos = pos.transpose()
neg= neg.transpose()

posp = pd.DataFrame(pos)
negp = pd.DataFrame(neg)

posp.insert(20, "Label", 1)
negp.insert(20, "Label", -1)
negp.shape
all_data =posp.append(negp)

all_data = all_data.values
np.random.shuffle(all_data)



In [449]:
#split data into 3 
test = all_data[:400]
val = all_data[400:800] 
train = all_data[800:] 

# save data
np.savetxt('DS1_test.csv',test,delimiter=',')
np.savetxt('DS1_val.csv',val,delimiter=',')
np.savetxt('DS1_train.csv',train,delimiter=',')

print( train.shape)
print( val.shape)
print( test.shape)


(3200, 21)
(400, 21)
(400, 21)


### 2 GDA
Estimation of parameters of the model: u0 (neg mean) , u1 (pos mean) , covariance matrix m

In [450]:
u0 = 0
u1= 0
num0=0
num1=0
tot=0

#Getting the mean vectors
for line in train:
    if line[20] == 1:   #positive
        u1+=line[:20]
        num1+=1
    else:
        u0+=line[:20]
        num0+=1
    tot+=1
        
u=pd.DataFrame(u1)
u0 =u0/num0
u1= u1/num1
p1 =num1/tot
p0 = num0/tot

s1=0
s0=0
for line in train:
    if line[20] == 1:   #positive
        vec= line[:20]
        vec = vec-u1
        vec= vec.reshape((1,20))
        tr = vec.reshape((20,1))
        temp = np.dot(tr , vec)  
        s1 += temp/num1
    else:              #negative 
        vec= line[:20]
        vec = vec-u0
        vec= vec.reshape((1,20))
        tr = vec.reshape((20,1))
        temp = np.dot(tr , vec)  
        #print(temp)
        s0 += temp/num0
#print(s1)
cov = (num1/tot)*s1 + (num0/tot)*s0


#### Coefficients

In [451]:
inv_cov = np.linalg.inv(cov)
w = np.dot(inv_cov, (u0-u1))


w0 = -0.5*(u0.transpose()).dot(inv_cov).dot(u0)+0.5*(u1.transpose()).dot(inv_cov).dot(u1)+np.log(p0/p1)
print("w : \n" ,w)
print("\n w0 : \n",w0)

w : 
 [ 14.07043598  -8.63642823  -5.5045595   -3.20634824  -9.60801489
  -4.31571584  16.70297422 -23.54632791 -28.80490041   9.13971621
 -12.88120057 -12.08863337  15.18266729  12.6824781   -5.46015667
  12.79328633  29.05398269  -6.54171831  -0.53833588  -4.7570663 ]

 w0 : 
 26.883504308974533


In [452]:
def prob0(x):
    temp= w.dot(x) +w0
    return 1/(1+np.exp(-temp))
def prob1(x):
    return (1- prob0(x))

#test
tp=0
tn=0
fp=0
fn=0

tot=0
for line in test:
    x = np.array(line[:20])
    label = line[20]
    pr1 = prob1(x)
    #print(label)
    pred =1
    if pr1 <0.5:   #if positive
        pred=0
    #print (pred)
    if(pred==1):
        if(label==1):  #true positive
            tp+=1
        else:
            fp+=1   #false positive
    else:
        if (label==-1):
            tn+=1
        else:
            fn+=1   #false neg
    tot+=1
    #if tot>10:
     #   break
            
acc=(tn+tp)/tot
prec = tp/(tp+fp)
recall = tp/(tp+fn)
F1 = 2*prec*recall /(prec+recall)
print("GDA performances: \n")            
print("The Accuracy is: ", acc)
print("The Precision is: ", prec)
print("The Recall is: " ,recall)
print("The F1 Measure is: ",F1)

GDA performances: 

The Accuracy is:  0.93
The Precision is:  0.937799043062201
The Recall is:  0.9289099526066351
The F1 Measure is:  0.9333333333333333


## 3. k-NN

In [463]:
def pred_class(x,k): # predict the class of x based on k-NN
    ave_val=0
    nn = []
    for i,row in enumerate(train):
        point = np.array(row[:-1])
        d = np.dot(x-point, x-point) #d^2
        
        if len(nn)<k:
            nn.append([i,d]) 
        else:
            nn.sort(key=lambda x: x[1]) #sort the  actual NN 
            if d<nn[k-1][1]:
                #del nn[-1]
                #nn.append([i,d])
                nn[k-1]= [i,d]

       
    ave_val = 0
    for line in nn:
        val = train[line[0]][-1] #get true val
        ave_val+=val
    
    ave_val/=k
    if ave_val<0:   # estimates the k-NN mean
        return 0
    else:
        return 1
    

In [454]:
# test accuracy, precision, recall, and F-measure


for k in range(1,12):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    tot = 0
    
    for row in val:
        tot+=1
        x = np.array(row[:-1])
        true_val = row[-1]
        
        choice = pred_class(x, k)
        if choice == 0:
            if true_val == -1: #true negative
                tn+=1
            else: #false negative
                fn+=1
        elif choice==1:
            if true_val == 1: #true positive
                tp+=1
            else: #false postive
                fp+=1


    acc = (tn+tp)/tot
    prec = tp/(tp+fp) 
    recall = tp/(tp+fn)
    F1 = 2*prec*recall /(prec+recall)
    print("k: ", k)
    print("The Accuracy is: ", acc)
    print("The Precision is: ", prec)
    print("The Recall is: " ,recall)
    print("The F1 Measure is: ",F1)
    print('\n')

k:  1
The Accuracy is:  0.5475
The Precision is:  0.4861878453038674
The Recall is:  0.5
The F1 Measure is:  0.4929971988795518


k:  2
The Accuracy is:  0.5075
The Precision is:  0.4620938628158845
The Recall is:  0.7272727272727273
The F1 Measure is:  0.565121412803532


k:  3
The Accuracy is:  0.55
The Precision is:  0.4898989898989899
The Recall is:  0.5511363636363636
The F1 Measure is:  0.5187165775401069


k:  4
The Accuracy is:  0.5325
The Precision is:  0.4795539033457249
The Recall is:  0.7329545454545454
The F1 Measure is:  0.5797752808988763


k:  5
The Accuracy is:  0.565
The Precision is:  0.5050505050505051
The Recall is:  0.5681818181818182
The F1 Measure is:  0.5347593582887701


k:  6
The Accuracy is:  0.54
The Precision is:  0.483739837398374
The Recall is:  0.6761363636363636
The F1 Measure is:  0.5639810426540285


k:  7
The Accuracy is:  0.5775
The Precision is:  0.5177664974619289
The Recall is:  0.5795454545454546
The F1 Measure is:  0.546916890080429


k:  8
Th

In [464]:
# 3.b For model selection, we choose the F1 measure criteria. 
# Then k=8 :


tp = 0
tn = 0
fp = 0
fn = 0
tot = 0

for k in range (8,9):    
    for row in test:
        tot+=1
        x = np.array(row[:-1])
        true_val = row[-1]
        
        choice = pred_class(x, k)
        if choice == 0:
            if true_val == -1: #true negative
                tn+=1
            else: #false negative
                fn+=1
        elif choice==1:
            if true_val == 1: #true positive
                tp+=1
            else: #false postive
                fp+=1

    acc = (tn+tp)/tot
    prec = tp/(tp+fp) 
    recall = tp/(tp+fn)
    F1 = 2*prec*recall /(prec+recall)
    print("k: ", k)
    print("The Accuracy is: ", acc)
    print("The Precision is: ", prec)
    print("The Recall is: " ,recall)
    print("The F1 Measure is: ",F1)
    print('\n')

k:  8
The Accuracy is:  0.54
The Precision is:  0.5594713656387665
The Recall is:  0.6018957345971564
The F1 Measure is:  0.5799086757990868




## 4 Muliple Gaussians

In [465]:
cols= range(20)
pos1 = np.loadtxt("./hwk2_datasets/DS2_c1_m1.txt", delimiter= ',', usecols=cols)
pos2 = np.loadtxt("./hwk2_datasets/DS2_c1_m2.txt", delimiter= ',', usecols=cols)
pos3 = np.loadtxt("./hwk2_datasets/DS2_c1_m3.txt", delimiter= ',', usecols=cols)

neg1 = np.loadtxt("./hwk2_datasets/DS2_c2_m1.txt", delimiter= ',', usecols=cols)
neg2 = np.loadtxt("./hwk2_datasets/DS2_c2_m2.txt", delimiter= ',', usecols=cols)
neg3 = np.loadtxt("./hwk2_datasets/DS2_c2_m3.txt", delimiter= ',', usecols=cols)

cov1= np.loadtxt("./hwk2_datasets/DS2_Cov1.txt", delimiter=',', usecols=cols)
cov2= np.loadtxt("./hwk2_datasets/DS2_Cov2.txt", delimiter=',', usecols=cols)
cov3= np.loadtxt("./hwk2_datasets/DS2_Cov3.txt", delimiter=',', usecols=cols)

mixture=[0.1 , 0.42, 0.48]

pos = []
np.random.seed(2)

for i in range(2000):
    rand = np.random.random()
    if(rand < mixture[0]):
        pos.append(np.random.multivariate_normal(pos1, cov1))
    elif(rand <mixture[0]+mixture[1]):
        pos.append(np.random.multivariate_normal(pos2, cov2))
    else:
        pos.append(np.random.multivariate_normal(pos3, cov3))

neg=[]        
for i in range(2000):
    rand = np.random.random()
    if(rand < mixture[0]):
        neg.append(np.random.multivariate_normal(neg1, cov1))
    elif(rand <mixture[0]+mixture[1]):
        neg.append(np.random.multivariate_normal(neg2, cov2))
    else:
        neg.append(np.random.multivariate_normal(neg3, cov3))   


posp = pd.DataFrame(pos)
negp = pd.DataFrame(neg)

posp.insert(20, "Label", 1)
negp.insert(20, "Label", -1)
all_data =posp.append(negp)
print (all_data.shape)
all_data = all_data.values
np.random.shuffle(all_data)


(4000, 21)


In [466]:
#split data into 3 
test = all_data[:400]
val = all_data[400:800] 
train = all_data[800:] 

# save data
np.savetxt('DS2_test.csv',test,delimiter=',')
np.savetxt('DS2_val.csv',val,delimiter=',')
np.savetxt('DS2_train.csv',train,delimiter=',')


## 5 Questions 2 & 3 with DS 2

In [467]:
#GDA

u0 = 0
u1= 0
num0=0
num1=0
tot=0

#Getting the mean vectors
for line in train:
    if line[20] == 1:   #positive
        u1+=line[:20]
        num1+=1
    else:
        u0+=line[:20]
        num0+=1
    tot+=1
        
u=pd.DataFrame(u1)
u0 =u0/num0
u1= u1/num1
p1 =num1/tot
p0 = num0/tot

s1=0
s0=0
for line in train:
    if line[20] == 1:   #positive
        vec= line[:20]
        vec = vec-u1
        vec= vec.reshape((1,20))
        tr = vec.reshape((20,1))
        temp = np.dot(tr , vec)  
        s1 += temp/num1
    else:              #negative 
        vec= line[:20]
        vec = vec-u0
        vec= vec.reshape((1,20))
        tr = vec.reshape((20,1))
        temp = np.dot(tr , vec)  
        #print(temp)
        s0 += temp/num0
#print(s1)
cov = (num1/tot)*s1 + (num0/tot)*s0


In [469]:
#Coefficients

inv_cov = np.linalg.inv(cov)
w = np.dot(inv_cov, (u0-u1))

w0 = -0.5*(u0.transpose()).dot(inv_cov).dot(u0)+0.5*(u1.transpose()).dot(inv_cov).dot(u1)+np.log(p0/p1)

print("w : \n" ,w)

print("\n w0 : \n",w0)

w : 
 [-0.06230741 -0.01771729 -0.12982761  0.06191243  0.0383158  -0.02825401
 -0.08601165 -0.01111571 -0.01205568  0.01182752  0.04258677  0.08571955
  0.05765785  0.10556944 -0.08429299  0.00581761 -0.02256758  0.05797166
  0.02994211  0.0098346 ]

 w0 : 
 -0.07563267236773956


In [470]:
def prob0(x):
    temp= w.dot(x) +w0
    return 1/(1+np.exp(-temp))
def prob1(x):
    return (1- prob0(x))

#test
tp=0
tn=0
fp=0
fn=0

tot=0
for line in test:
    x = np.array(line[:20])
    label = line[20]
    pr1 = prob1(x)
    #print(label)
    pred =1
    if pr1 <0.5:   #if positive
        pred=0
    #print (pred)
    if(pred==1):
        if(label==1):  #true positive
            tp+=1
        else:
            fp+=1   #false positive
    else:
        if (label==-1):
            tn+=1
        else:
            fn+=1   #false neg
    tot+=1
    #if tot>10:
     #   break
            
acc=(tn+tp)/tot
prec = tp/(tp+fp)
recall = tp/(tp+fn)
F1 = 2*prec*recall /(prec+recall)
print("GDA performances : \n")            
print("The Accuracy is: ", acc)
print("The Precision is: ", prec)
print("The Recall is: " ,recall)
print("The F1 Measure is: ",F1)

GDA performances : 

The Accuracy is:  0.4625
The Precision is:  0.44075829383886256
The Recall is:  0.48947368421052634
The F1 Measure is:  0.4638403990024938


In [444]:
# 5.2 k-NN

for k in range(1,12):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    tot = 0
    
    for row in val:   #model selection on validaton set
        tot+=1
        x = np.array(row[:-1])
        true_val = row[-1]
        #print("true =", true_val)
        #nn = get_nn(k, x)
        #choice = define_class(k,nn)
        choice = pred_class(x, k)
        if choice == 0:
            if true_val == -1: #true negative
                tn+=1
            else: #false negative
                fn+=1
        elif choice==1:
            if true_val == 1: #true positive
                tp+=1
            else: #false postive
                fp+=1


    acc = (tn+tp)/tot
    prec = tp/(tp+fp) 
    recall = tp/(tp+fn)
    F1 = 2*prec*recall /(prec+recall)
    print("k: ", k)
    print("The Accuracy is: ", acc)
    print("The Precision is: ", prec)
    print("The Recall is: " ,recall)
    print("The F1 Measure is: ",F1)
    print('\n')

k: 1
The Accuracy is:  0.515
The Precision is:  0.5196078431372549
The Recall is:  0.5247524752475248
The F1 Measure is:  0.5221674876847291


k: 2
The Accuracy is:  0.4975
The Precision is:  0.5016393442622951
The Recall is:  0.7574257425742574
The F1 Measure is:  0.6035502958579881


k: 3
The Accuracy is:  0.53
The Precision is:  0.5339805825242718
The Recall is:  0.5445544554455446
The F1 Measure is:  0.5392156862745098


k: 4
The Accuracy is:  0.5125
The Precision is:  0.5127272727272727
The Recall is:  0.698019801980198
The F1 Measure is:  0.5911949685534591


k: 5
The Accuracy is:  0.52
The Precision is:  0.5240384615384616
The Recall is:  0.5396039603960396
The F1 Measure is:  0.5317073170731708


k: 6
The Accuracy is:  0.51
The Precision is:  0.5108695652173914
The Recall is:  0.698019801980198
The F1 Measure is:  0.5899581589958159


k: 7
The Accuracy is:  0.5075
The Precision is:  0.5119617224880383
The Recall is:  0.5297029702970297
The F1 Measure is:  0.5206812652068127


k

In [471]:
for k in range(2,3):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    tot = 0
    
    for row in test:   #testing
        tot+=1
        x = np.array(row[:-1])
        true_val = row[-1]
        #print("true =", true_val)
        #nn = get_nn(k, x)
        #choice = define_class(k,nn)
        choice = pred_class(x, k)
        if choice == 0:
            if true_val == -1: #true negative
                tn+=1
            else: #false negative
                fn+=1
        elif choice==1:
            if true_val == 1: #true positive
                tp+=1
            else: #false postive
                fp+=1


    acc = (tn+tp)/tot
    prec = tp/(tp+fp) 
    recall = tp/(tp+fn)
    F1 = 2*prec*recall /(prec+recall)
    print("k: ", k)
    print("The Accuracy is: ", acc)
    print("The Precision is: ", prec)
    print("The Recall is: " ,recall)
    print("The F1 Measure is: ",F1)
    print('\n')

k:  2
The Accuracy is:  0.4975
The Precision is:  0.4816053511705686
The Recall is:  0.7578947368421053
The F1 Measure is:  0.5889570552147239


